# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../WeatherPy/output_data/weather_data.csv")
vacation_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,albany,75,AU,1580358347,93,-35.02,117.89,59.00,12.75
1,1,broome,40,AU,1580358457,63,-17.97,122.23,91.40,13.87
2,2,rikitea,100,PF,1580358606,78,-23.12,-134.97,78.46,14.70
3,3,butaritari,80,KI,1580358606,73,3.07,172.79,85.08,7.85
4,4,atuona,2,PF,1580358607,79,-9.80,-139.03,80.71,14.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = vacation_df[['Lat', 'Lng']]
weight = vacation_df['Humidity'].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_temp = vacation_df.loc[(vacation_df['Max Temp'] <80) & (vacation_df['Max Temp'] >70)]
ideal_wind = ideal_temp.loc[ideal_temp['Wind Speed'] <10]
ideal_cloud = ideal_wind.loc[ideal_wind['Cloudiness']==0] 
ideal_cloud

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
18,19,jamestown,0,SH,1580358609,78,-15.94,-5.72,74.05,8.63
245,274,igatpuri,0,IN,1580358991,24,19.70,73.55,79.57,1.19
490,552,mormugao,0,IN,1580359391,69,15.40,73.80,75.20,8.05
508,572,paredon,0,MX,1580359453,74,16.03,-93.87,77.74,9.40
532,597,partur,0,IN,1580359457,41,19.60,76.22,75.60,6.53
533,600,visavadar,0,IN,1580359457,26,21.38,70.68,73.42,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(ideal_cloud, columns=['City', 'Cloudiness', 'Country', 'Date',
                                             'Humidity', 'Lat', 'Lng', 'Max Temp',
                                             'Wind Speed'])
hotel_df['Hotel Name']=''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
18,jamestown,0,SH,1580358609,78,-15.94,-5.72,74.05,8.63,
245,igatpuri,0,IN,1580358991,24,19.70,73.55,79.57,1.19,
490,mormugao,0,IN,1580359391,69,15.40,73.80,75.20,8.05,
508,paredon,0,MX,1580359453,74,16.03,-93.87,77.74,9.40,
532,partur,0,IN,1580359457,41,19.60,76.22,75.60,6.53,
533,visavadar,0,IN,1580359457,26,21.38,70.68,73.42,5.99,


In [7]:
params = {"radius": 5000,
          "types": "restaurant",
          "key": g_key
         }

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    
    params['location'] = f"{lat},{lon}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    names = requests.get(url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = names["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing result... skipping.")

In [8]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
18,jamestown,0,SH,1580358609,78,-15.94,-5.72,74.05,8.63,Richards Travel Lodge
245,igatpuri,0,IN,1580358991,24,19.70,73.55,79.57,1.19,Manas Resort With Petting Zoo
490,mormugao,0,IN,1580359391,69,15.40,73.80,75.20,8.05,Hotel Anantashram
508,paredon,0,MX,1580359453,74,16.03,-93.87,77.74,9.40,Restaurant Zalatiel
532,partur,0,IN,1580359457,41,19.60,76.22,75.60,6.53,Jyoti Tarodkar
533,visavadar,0,IN,1580359457,26,21.38,70.68,73.42,5.99,Hotel ramnath


In [9]:
hotel_df.to_csv('output_data/Hotels.csv', index=False)

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
#creating marker locations
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))